# Plotting Methods for All Analyses

In [18]:
%pylab inline
import sys
sys.path.insert(0, '../modules')

Populating the interactive namespace from numpy and matplotlib


In [ ]:
from cylinder import TrackCenters

## Default Size Options

In [17]:
dot_size = 40

In [35]:
# Declare Globally 
wire_rhos, wire_phis = None, None
wire_xs, wire_ys = None, None
n_wires = 4482
plot_cydet = None

def plot_set_cydet(cydet_instance):
    """
    Setup wire locations in x,y and r, theta
    """
    plot_cydet = cydet_instance
    wire_rhos, wire_phis = plot_cydet.get_points_rhos_and_phis()
    wire_xs, wire_ys = plot_cydet.get_points_xs_and_ys()
    n_wires = plot_cydet.n_points

In [28]:
# Declare Globally 
track_rhos, track_phis = None, None
track_xs, track_ys = None, None
n_centres = None
plot_hough = None

def plot_set_hough(hough_instance):
    """
    Setup track centre locations in x,y and r, theta
    """
    plot_hough = hough_instance
    track_rhos, track_phis = hough_instance.track.get_points_rhos_and_phis()
    track_xs, track_ys = hough_instance.track.get_points_xs_and_ys()
    n_centres = hough_instance.track.n_points

In [15]:
def plot_set_font(size=15):
    """
    Default Font for all plots 
    """
    font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : size}
    # Enable LaTeX support in fonts
    matplotlib.rc('font', **font)

In [32]:
def plot_setup_for_detector(size=(15, 15)):
    """
    Default plot for event display
    """
    plot_set_font()
    fig = plt.figure(1, figsize=size)
    fig.figsize(*size)
    fig.axes(polar=True)
    fig.ylim(0, 85)
    return fig

In [ ]:
def plot_get_hits(labels):
    """
    Gets the signal and background hits for an event
    
    :param labels: Labels from an event
    
    :return: signal hits, background hits
    """
    # Ensure the event is the right number of wires
    assert len(labels) == n_wires,\
        "Number of input wires is {}, real number is {}".format(len(labels), n_wires)
    # Get the signal hits
    sig_hits = np.where(labels == 1)
    bkg_hits = np.where(labels == 2)
    return sig_hits, bkg_hits

In [34]:
def plot_output(labels, size=dot_size*np.ones(n_wires), sig_color="blue", bkg_color="red", figsize=(12,14)):
    """
    Draw the output of a classifier by scaling the hits to the size of the output.
    
    :param labels:    Triple labels to tell signal from background from empty
    :param size:      Size of all hits on every wire, this includes empty wires
    :param sig_color: Colour of signal hits, default blue
    :param bkg_color: Colour of background hits, default red
    
    :return: axis of plot, figure of plot
    """
    # Set the default font
    plot_set_font()
    # Get the hits from the labels
    sig_hits, bkg_hits = plot_get_hits(labels)
    # Get axis and figure
    fig = plt.figure(1, figsize=figsize)
    axs = fig.add_subplot(111, projection='polar')
    axs.set_ylim([0,85])
    # Set theta ticks at 45 degrees
    thetaticks = np.arange(0, 360, 45)
    # Set ticks further out from edge
    axs.set_thetagrids(thetaticks, frac=1.05)
    # Set radial ticks
    axs.set_yticks(np.arange(10,81,20))
    # Plot all wires
    print wire_phis, wire_rhos
    axs.scatter(wire_phis, wire_rhos, marker='.', s=5, alpha=0.2)
    # Plot signal hits
    axs.scatter(wire_phis[sig_hits],wire_rhos[sig_hits], s=size[sig_hits], 
                marker='o', color=sig_color)
    # Plot background hits
    axs.scatter(wire_phis[bkg_hits],wire_rhos[bkg_hits], s=size[bkg_hits], 
                marker='o', color=bkg_color)
    # Return axis and figure
    return axs, fig

In [ ]:
def plot_add_outlines(labels, sig_color="blue", bkg_color="red", size=dot_size):
    """
    Adds in outlines of original locations of hits
    
    :param labels:    Triple labels to tell signal from background from empty
    :param size:      Size of all hits on every wire, this includes empty wires
    :param sig_color: Colour of signal hits, default blue
    :param bkg_color: Colour of background hits, default red
    """
    # Get the hits from the labels
    sig_hits, bkg_hits = plot_get_hits(labels)
    # Add the outlines of the signal and background
    gca().scatter(wire_phis[sig_hits],wire_rhos[sig_hits], s=size,
            marker='o', edgecolors=sig_color, facecolors='none',  lw = 0.5)
    gca().scatter(wire_phis[bkg_hits],wire_rhos[bkg_hits], s=size,
            marker='o', edgecolors=bkg_color, facecolors='none',  lw = 0.5)

In [ ]:
def plot_add_circle(x, y, radius, color="green", lw=1., spread=0, l_alpha=1., s_alpha=0.025):
    """
    Add a circle to our plot
    
    :param x:        x location of circle centre
    :param y:        y location of circle centre
    :param radius:   radius of circle
    :param color:    color of circle
    :param lw:       line width of circle
    :param spread:   spread of circle, symmetric
    :param l_alpha:  overall normalization on weight of line
    :param s_alpha:  overall normalization on weight of spread
    """
    ## TODO check gca() usage here
    plot_circle = plt.Circle((x, y), radius, transform=gca().transData._b,
            color=color, fill=False, alpha=l_alpha, lw=lw)
    gca().add_artist(plot_circle)
    # Add a spread if need be
    if spread!=0:
        plot_spread = plt.Circle((x, y), radius,
                transform=gca().transData._b, color="green",
                alpha=s_alpha, fill=False, lw=spread)
        gca().add_artist(plot_spread)

In [ ]:
def plot_add_transform(index, inverse=False, color="green", l_alpha=1., s_alpha=0.025,
                      prob_spread=True):
    """
    Add a hough transform overlay, assumes forward transform unless told otherwise
    
    :param index:        index of wire or track centre
    :param inverse:      If true, plot from track centre to wires
    :param color:        color of circle
    :param l_alpha:      overall normalization on weight of line
    :param s_alpha:      overall normalization on weight of spread
    :param prob_spread:  Call the actual correpsondence function to determine the shading
                         of the probability spread
    """
    # Define the signal radius
    r_sig = plot_hough.sig_rho
    r_max = plot_hough.sig_rho_max 
    r_min = plot_hough.sig_rho_min
    # Check if we are mapping forwards or inverse
    if inverse:
        plot_x = track_xs[index]
        plot_y = track_ys[index]
    else:
        plot_x = wire_xs[index]
        plot_y = wire_ys[index]
        
    # Add the target radius circle
    plot_add_circle((plot_x, plot_y), r_sig, color=color, l_alpha=l_alpha, s_alpha=s_alpha)
    # Add the spread around the target as 10 shaded circles at varying distances
    for r in np.linspace(r_min, r_max, 10):
        # Check if we're actually calling the function
        if prob_spread == True:
            shading = s_alpha*plot_hough.dist_prob(r)/plot_hough.dist_prob(radius)
        # Define it as a constant weight otherwise
        else: 
            shading = s_alpha/2.
        # Add the spread
        plot_add_circle((plot_x, plot_y), r, color=color, l_alpha=shading, lw=3)

In [29]:
def plot_add_tracks(tolerance=0.001, size=(dot_size/5.)*np.ones(n_wires), 
                    color="DarkOrange", add_transform=False):
    """
    Add track centres as orange points
    
    :param tolerance:     Relative size of smallest drawn track centre to maximum track
                          centre
    :param size:          Size of all track centres
    :param color:         Colour of track centres
    :param add_transform: Adds default transform if true
    """
    # Find the tracks above tolerance of the max
    draw_tracks = np.where(size > np.amax(size)*tolerance)[0]
    # Draw these tracks
    gca().scatter(track_phis[draw_tracks] ,track_rhos[draw_tracks],
            s=track_size[draw_tracks], marker='o', color=color, zorder=10)
    # If we want transforms, add them too
    if add_transform:
        for trk in draw_tracks:
            plot_add_transform()

In [ ]:
def plot_norm_size(bdt_guess, size=dot_size):
    return (float(size)*bdt_guess)/np.amax(bdt_guess)